In [2]:
import os
import numpy as np
import pydicom
import cv2
import matplotlib.pyplot as plt
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
import torch.optim as optim

In [3]:
# Add targets
targets_df = pd.read_csv('./train.csv')

mapping = {
    'normal/mild': 1,
    'moderate': 2,
    'severe': 3
}
def standardize_and_map(column, mapping):
    return column.str.lower().str.strip().map(mapping)

# List of columns to apply the transformation
columns_to_transform = ['spinal_canal_stenosis_l1_l2',
       'spinal_canal_stenosis_l2_l3', 'spinal_canal_stenosis_l3_l4',
       'spinal_canal_stenosis_l4_l5', 'spinal_canal_stenosis_l5_s1',
       'left_neural_foraminal_narrowing_l1_l2',
       'left_neural_foraminal_narrowing_l2_l3',
       'left_neural_foraminal_narrowing_l3_l4',
       'left_neural_foraminal_narrowing_l4_l5',
       'left_neural_foraminal_narrowing_l5_s1',
       'right_neural_foraminal_narrowing_l1_l2',
       'right_neural_foraminal_narrowing_l2_l3',
       'right_neural_foraminal_narrowing_l3_l4',
       'right_neural_foraminal_narrowing_l4_l5',
       'right_neural_foraminal_narrowing_l5_s1',
       'left_subarticular_stenosis_l1_l2', 'left_subarticular_stenosis_l2_l3',
       'left_subarticular_stenosis_l3_l4', 'left_subarticular_stenosis_l4_l5',
       'left_subarticular_stenosis_l5_s1', 'right_subarticular_stenosis_l1_l2',
       'right_subarticular_stenosis_l2_l3',
       'right_subarticular_stenosis_l3_l4',
       'right_subarticular_stenosis_l4_l5',
       'right_subarticular_stenosis_l5_s1']


In [4]:
# Apply the function to the specified columns
for column in columns_to_transform:
    targets_df[column] = standardize_and_map(targets_df[column], mapping)

In [5]:
test_df = targets_df[targets_df['study_id'] == 10728036]
test_df

,study_id,spinal_canal_stenosis_l1_l2,spinal_canal_stenosis_l2_l3,spinal_canal_stenosis_l3_l4,spinal_canal_stenosis_l4_l5,spinal_canal_stenosis_l5_s1,left_neural_foraminal_narrowing_l1_l2,left_neural_foraminal_narrowing_l2_l3,left_neural_foraminal_narrowing_l3_l4,left_neural_foraminal_narrowing_l4_l5,...,left_subarticular_stenosis_l1_l2,left_subarticular_stenosis_l2_l3,left_subarticular_stenosis_l3_l4,left_subarticular_stenosis_l4_l5,left_subarticular_stenosis_l5_s1,right_subarticular_stenosis_l1_l2,right_subarticular_stenosis_l2_l3,right_subarticular_stenosis_l3_l4,right_subarticular_stenosis_l4_l5,right_subarticular_stenosis_l5_s1
4,10728036,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0


In [6]:
stacked_tensors = {'10728036': 1, '11340341': 2, '11943292': 3, '13317052': 4, '22191399': 5, '26342422': 6, '29931867': 7, '33736057': 8, '4003253': 9, '4646740': 10, '7143189': 11, '8785691': 12}

In [7]:
# Store targets in a dict as torch tensors and rashape them

targets_tensors = {}

for key, arr in stacked_tensors.items():
    target = targets_df[targets_df['study_id'] == int(key)]
    target_study_id = target.iloc[:, :1]
    target_values_int = np.array(target.iloc[:, 1:]).flatten().astype(int) - 1

    # Convert the target array to one-hot encoding with three classes
    num_classes = 3
    one_hot_targets = np.eye(num_classes)[target_values_int]

    # Reshape the one-hot encoded target array to match the desired output shape (25, 3)
    reshaped_targets = one_hot_targets.reshape(-1, num_classes)

    # Convert the numpy array to a torch tensor
    tensor_targets = torch.from_numpy(reshaped_targets)

    targets_tensors[target_study_id.iloc[0, 0]] = tensor_targets

In [8]:
see = targets_tensors[13317052]
see.shape

torch.Size([25, 3])

In [9]:
# class CustomCNN(nn.Module):
#     def __init__(self):
#         super(CustomCNN, self).__init__()

#         # Convolutional and Pooling layers
#         self.conv1 = nn.Conv2d(in_channels=4, out_channels=16, kernel_size=3, stride=1, padding=1)
#         self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
#         self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)

#         # Linear layers
#         self.fc1 = nn.Linear(32 * 8 * 8, 512)  # Adjust the input features to match the output of your last conv layer
#         self.fc2 = nn.Linear(512, 25*3)  # The output should match the flattened target tensor shape

#     def forward(self, x):
#         # Define the forward pass
#         x = F.relu(self.conv1(x))

#         x = F.relu(self.conv2(x))
#         # ... add more layers as needed ...
#         x = torch.flatten(x, 1)  # Flatten the tensor for the fully connected layer
#         x = F.relu(self.fc1(x))
#         x = self.fc2(x)
#         return x

In [10]:
class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        # self.layer1 = nn.Sequential(
        #     nn.Conv2d(in_channels=4, out_channels=16, kernel_size=5, stride=1, padding=2),
        #     nn.MaxPool2d(kernel_size=(8, 2), stride=(8, 2)))  # Large pooling to reduce height quickly
        # self.layer2 = nn.Sequential(
        #     nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2),
        #     nn.MaxPool2d(kernel_size=(4, 2), stride=(4, 2)))  # Smaller pooling
        
        self.conv1 = nn.Conv2d(in_channels=4, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.pool = nn.MaxPool2d(kernel_size=(8, 2), stride=(8, 2))  # Large pooling to reduce height quickly
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.pool2 = nn.MaxPool2d(kernel_size=(4, 2), stride=(4, 2))  # Smaller pooling

        self.fc1 = nn.Linear(32*406*56, 25*3)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        # x = self.fc1(x)
        # x = x.view(x.size(0), -1)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        x = x.reshape(x.shape[0], 25, 3)

        return x

In [11]:
model = CustomCNN()

In [12]:
x = torch.randn(2, 4, 12992, 224)
print(model(x).shape)

torch.Size([2, 25, 3])


In [26]:
targets = torch.randn(2, 3, 25)
# targets = targets.view(2, -1)
# x = x.reshape(x.shape[0], 25, 3)
targets = targets.reshape(targets.shape[0], -1)
targets.shape

torch.Size([2, 75])

In [28]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2)
        # Additional layers...
        self.fc = nn.Linear(in_features=28 , out_features=75) # Adjust in_features according to previous layer output

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        # Additional layers...
        x = x.view(x.size(0), -1) # Flatten the tensor for the fully connected layer
        x = self.fc(x)
        return x.view(-1, 25, 3) # Reshape to match the target shape

# Instantiate the model
model = CNNModel()

# Define the loss function and optimizer
criterion = nn.BCEWithLogitsLoss() # Suitable for multi-label classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Example input batch
inputs = torch.randn(2, 1, 28, 28) # Example input shape, adjust as necessary
targets = torch.randint(0, 2, (2, 25, 3)).float() # Random targets for demonstration

# Training loop
for epoch in range(5):
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()

    print(f'Epoch [{epoch+1}/{5}], Loss: {loss.item()}')

RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x6272 and 28x75)